In [1]:
import psycopg2
import pandas as pd

import re

import numpy as np
import nltk
from nltk.corpus import stopwords
# import nltk.tokenize as nt
# from nltk.tokenize import RegexpTokenizer

from textblob import blob, Blobber, TextBlob, Sentence, Word, WordList, tokenizers, sentiments, taggers, parsers
#from textblob_aptagger import PerceptronTagger

In [2]:
con = psycopg2.connect(dbname="skillsdb",host="dw-instance.cbrlhmbtfrqg.eu-west-2.redshift.amazonaws.com"
                ,port="5439",user="masteruser", password="Ehgh1363")
curs = con.cursor()

In [3]:
# total number of CVs

pd.read_sql_query('''select count(distinct user_id) from cv''',con)

,count
0,5049


In [4]:
data = pd.read_sql_query('''select distinct cv.user_id, cv_section_attribute.name,
cv.value_char, cv.value_timestamp from cv_section_attribute 
left join cv on cv_section_attribute.id=cv.cv_section_attribute_id''',con)

In [5]:
# data['value_char'] = data['value_char'].map(lambda x: x.strip() if pd.notnull(x) else x)

In [6]:
data_valid = data[data['name'].isin(['locale','name','summary','headline',
                                     'degree','school','admit_year','grad_year',
                                     'company',  'title',  'work_location',  
                                     'start_date','end_date', 'description',
                                     'award',
                                     'publication', 
                                     'additional_info', 
                                     'skill'])]

In [7]:
# cv_headlines = data[data['name'] == 'headline'][['user_id','value_char']]
# cv_degrees = data[data['name'] == 'degree'][['user_id','value_char']]
# cv_schools = data[data['name'] == 'school'][['user_id','value_char']]
# cv_locales = data[data['name'] == 'locale'][['user_id','value_char']]
# cv_summaries = data[data['name'] == 'summary'][['user_id','value_char']]
# cv_companies = data[data['name'] == 'company'][['user_id','value_char']]
# cv_titles = data[data['name'] == 'title'][['user_id','value_char']]
# cv_work_locations = data[data['name'] == 'work_location'][['user_id','value_char']]
# cv_descriptions = data[data['name'] == 'work_description'][['user_id','value_char']]
# cv_awards = data[data['name'] == 'award'][['user_id','value_char']]
# cv_skills = data[data['name'] == 'skill'][['user_id','value_char']]
# cv_add_info = data[data['name'] == 'additional_info'][['user_id','value_char']]

In [8]:
# cv_headlines.merge(
#     cv_summaries, how='outer', on='user_id').merge(
#     cv_locales, how='outer', on='user_id').merge(cv_awards, how='outer', on='user_id').merge(cv_add_info, how='outer', on='user_id')

In [9]:
## Tokenization parameters

stopWords = stopwords.words('english')
tokenizer_prefs = {
    'tokenizer' : nltk.tokenize.PunktSentenceTokenizer(),
#     'token_format' : 'stem',
    'spell_correct' : False,
    'np_extract': None,
    'pos_tagger': None,
    'analyzer': None,
    'classifier': None, 
    'clean_html': False
}

In [10]:
def clean_data(**kwargs):
    '''
    Cleans text data by:
    1.  force lowercase
    2.  remove non-ascii chars
    3.  standardize whitespace
    4.  remove digits
    5.  remove control characters
    6.  remove URL patterns
    '''
    df = pd.DataFrame(data_valid)
    
    try:
        df['value_char'] = data_valid['value_char'].dropna().map(lambda x: "".join(i for i in x.strip().lower() if ord(i)<128))
    except UnicodeDecodeError:
        print(UnicodeDecodeError)
        df['value_char'] = data_valid['value_char'].dropna().map(lambda x: x.strip().lower())


    url_pattern = "((http|ftp|https):\/\/)?[\w\-_]+(\.[\w\-_]+)+([\w\-\.,@?^=%&amp;:/~\+#]*[\w\-\@?^=%&amp;/~\+#])?"

    re_URL = re.compile(url_pattern)
#     re_TAG = re.compile("(<[phl]>)", re.IGNORECASE)
    re_WS = re.compile("/[^\S\n]/")
#     re_DIGIT = re.compile("\d")
    re_CTRL = re.compile("[\x00-\x11\x03-\x1F]+")
    re_HI = re.compile("[\x80-\xFF]+")
    re_NWC = re.compile("[!;<>?{}\/~`#=@#$%^&*()_+]")
    
    df['value_char'] = df['value_char'].map(lambda x: re_HI.sub(' ', x) if type(x) == str else None)
    df['value_char'] = df['value_char'].map(lambda x: re_CTRL.sub(' ', x) if type(x) == str else None)
    df['value_char'] = df['value_char'].map(lambda x: re_URL.sub(' ', x) if type(x) == str else None)
#     data[prefix] = data[prefix].map(lambda x: re_DIGIT.sub(' ', x))
    df['value_char'] = df['value_char'].map(lambda x: re_WS.sub(' ', x) if type(x) == str else None)        
    df['value_char'] = df['value_char'].map(lambda x: re_NWC.sub(' ', x) if type(x) == str else None)
    

    # create a blon using TextBlob
    tokenizer = kwargs['tokenizer']
    pos_tagger = kwargs['pos_tagger']
    analyzer = kwargs['analyzer']
    classifier = kwargs['classifier']
    np_extract = kwargs['np_extract']
    
    df['value_char'] = df['value_char'].map(lambda l: TextBlob(l,
                                          tokenizer=tokenizer,
                                           np_extractor=np_extract,
                                           pos_tagger=pos_tagger,
                                           analyzer=analyzer) if l is not None else None)

    df_sentences = pd.DataFrame(df)
    # tokenize the document into sentences from blob object
    df_sentences['value_char'] = df['value_char'].map(lambda s: s.sentences if s is not None else None)
    
    df_words = pd.DataFrame(df_sentences)
    # tokenize each sentence into words
#     df_words = df_sentences['value_char'].map(lambda l: map(lambda w: w.strip().words if w is not None and len(w)>1 else None, l))
    df_words['value_char'] = df_sentences['value_char'].dropna().map(lambda l: (w.strip().words for w in l if w is not None and len(w)>1))
    
    
    return df_words

In [11]:
df = clean_data(**tokenizer_prefs)

In [656]:
# tokenizer = tokenizer_prefs['tokenizer']
# pos_tagger = tokenizer_prefs['pos_tagger']
# analyzer = tokenizer_prefs['analyzer']
# classifier = tokenizer_prefs['classifier']
# np_extract = tokenizer_prefs['np_extract']

# b = df['value_char'].map(lambda l: TextBlob(l,
#                                       tokenizer=tokenizer,
#                                        np_extractor=np_extract,
#                                        pos_tagger=pos_tagger,
#                                        analyzer=analyzer,
#                                        classifier=classifier) if l is not None else None)

In [11]:
# data_valid = data_valid.reset_index(drop=True)

In [12]:
# ## Tokenization

# # initialize the tokenizer

# tokenizer = nltk.tokenize.PunktSentenceTokenizer()

# # tokenize data

# data_clean['value_char'] = data_clean['value_char'].map(lambda x: tokenizer.tokenize(x.strip()))
# data_clean.head()

In [13]:
# word_tokenizer = nltk.tokenize.word_tokenize()
# data_clean['value_char'] = data_clean['value_char'].map(lambda x: nltk.tokenize.word_tokenize(x))

In [14]:
# ## Stemming

# # initialize the stemmer

# stemmer = nltk.stem.PorterStemmer()
# # data_clean['value_char'].map(lambda x: (i for i in x))

In [15]:
# lemmatize = nltk.WordNetLemmatizer

In [45]:
# data_blobs = pd.DataFrame()
# data_blobs['value_char'] = create_blob('value_char', **tokenizer_prefs)
# data_blobs.value_char[0:10]

In [160]:
# def tokenize_sentences(prefix):
# #     tokenizer = kwargs['tokenizer']
# #     normalizer = kwargs['token_format']

#     # tokenize the document into sentences from blob object
#     sentences = data_blobs[prefix].map(lambda s: s.sentences)

#     return sentences

In [114]:
# data_sentence_tokenized = pd.DataFrame()
# data_sentence_tokenized['value_char'] = tokenize_sentences('value_char')
# data_sentence_tokenized.value_char[0:10]

In [162]:
# def tokenize_words(prefix):
# #     tokenizer = kwargs['tokenizer']
# #     normalizer = kwargs['token_format']

#     # tokenize each sentence into words
#     # trim token whitespaces
#     # eliminate tokens of character length 1
#     #words = self.data[prefix].map(lambda w: w.strip().tokens if len(w)>1 else None)

#     words = data_sentence_tokenized[prefix].map(lambda l: map(lambda w: w.strip().words if len(w)>1 else None, l))

#     return words

In [163]:
# data_word_tokenized = pd.DataFrame()
# data_word_tokenized = tokenize_words('value_char')

In [164]:
# def normalize_data(data_word_tokenized):
# #     tokenizer = kwargs['tokenizer']
# #     normalizer = kwargs['token_format']
# #     spelling = kwargs['spell_correct']
    
#     data_normalized = data_word_tokenized.map(lambda l: map(lambda w: w.singularize() if not isinstance(w, type(None)) and len(w)>1 else None, l))
# #     data_normalized = data_normalized.dropna()
    
#     # filter out 'bad' words, normalize good ones
#     # w if w not in self.stopWords else wl.remove(w)
# #     data_normalized = data_normalized.map(lambda l: map(lambda wl: map(lambda w: wl.remove(w) if w in stopWords and not isinstance(w, type(None)) else w, wl), l))
# #     data_normalized = data_normalized.map(lambda l: map(lambda wl: map(lambda w: wl.remove(w) if w == '\'s' and not isinstance(w, type(None) else w, wl), l))
# #     data_normalized = data_normalized.map(lambda l: map(lambda wl: map(lambda w: wl.remove(w) if w == '\'d' and not isinstance(w, type(None) else w, wl), l))
# #     data_normalized = data_normalized.dropna()

#      # remove tokens with length 1
# #     ree = re.compile(r'(\'\w)')
# #     rlen = len(data_normalized)
# #     tmp = data_normalized.copy()
# # #     tmp.reset_index(drop=True)
# #     for indx in tmp.index:
# #          wl_coll = list()
# #          for i in normalize(data_word_tokenized,**tokenizer_prefs):
# #             for j in i:
# #                 if not isinstance(j, type(None)):
# #                     for k in j:
# #                         if not isinstance(k, type(None)):
# #                             if re.match(ree, i):
# #                                 ree.sub('', i)
# #                             if len(i.strip().strip('.').strip(',')) > 1:
# #                                 wl.append((i))
# #                     wl_coll.append(WordList(wl))
# #             data_normalized[indx] = wl_col
# #             del tmp

#     # stemming
# #     data_normalized = data_normalized.map(lambda l: map(lambda wl: map(lambda w: nltk.stemmer.stem(w) if w in wl and not isinstance(w, types.NoneType) else wl.remove(w), wl), l))

# #     data_word_tokenized= tokenize_words('value_char', **tokenizer_prefs)
    
#     return data_normalized

In [165]:
# data_normalized = normalize_data(data_word_tokenized)

In [146]:
# if not isinstance(data_normalized, type(None)):
#     for i in data_normalized:
#         if not isinstance(i, type(None)):
#             for j in i:
#                 if not isinstance(j, type(None)):
#                     for k in j:
#                         if not isinstance(k, type(None)) and k not in stopWords and k != '\'s' and k != '\'d':
#                             print(k)

In [665]:
def build_vocab(target):
    vocab = set()
#     if not isinstance(df, type(None)):
    for token in df.value_char:
        if not isinstance(token, type(None)) and type(token) !=float:
            for sentence in token:
                if not isinstance(sentence, type(None)) and type(sentence) !=float:
                    for word in sentence:
#                         if not isinstance(word, type(None)) and word not in stopWords and word != '\'s' and word != '\'d':
                         if not isinstance(word, type(None)) and word != '\'s' and word != '\'d':
                            vocab.add(word)
            
    if target:
        w2i = {w: np.int32(i+2) for i, w in enumerate(vocab)}
        w2i['<s>'], w2i['</s>'] = np.int32(0), np.int32(1)
    else:
        w2i = {w: np.int32(i) for i, w in enumerate(vocab)}

    return w2i

In [666]:
w2i = build_vocab(True)

In [667]:
w2i

{'omdurmanuiversity2011': 2,
 'interviewereffective': 1018,
 ...}

In [166]:
def encode(data_normalized, w2i):
    encoded_sentence = []
    if not isinstance(data_normalized, type(None)):
        for token in data_normalized:
            if not isinstance(token, type(None)):
                for sentence in token:
                    if not isinstance(sentence, type(None)):
                        for w in sentence:
                            try:
                                encoded_sentence.append(w2i[w])
                            except Exception:
                                pass
    return encoded_sentence

In [157]:
encode(data_normalized, w2i)

[34587,
 ...]

In [167]:
def load_data(data_normalized, vocab=None, w2i=None, target=True):
    if vocab is None and w2i is None:
        w2i = build_vocab(data_normalized, target)

    s = []
    data = []
    if not isinstance(data_normalized, type(None)):
        for token in data_normalized:
            if not isinstance(token, type(None)):
                for sentence in token:
                    if not isinstance(sentence, type(None)):
                        for w in sentence:
                            s.append(w)
        if target:
            s = ['<s>'] + s + ['</s>']
        enc = encode(s, w2i)
        data.append(enc)
    i2w = {i: w for w, i in w2i.items()}
    return data, w2i, i2w

In [168]:
load_data(data_normalized,w2i=w2i,target=False)

([[41997,
   ...]],
 {'hypothesi': 2,
  ...},
 {0: '<s>',
  1: '</s>',
  2: 'hypothesi',
  3: 'etching',
  4: 'orthopedist',
  5: 'premiere-based',
  6: 'perday',
  7: 'andposition',
  8: 'outlay',
  9: 'dorado',
  10: 'lorry',
  11: 'semisolid',
  12: '2013tinatus',
  13: 'sister',
  14: 'became',
  15: 'geometryal-qud',
  16: 'vmsimplementing',
  17: 'sh',
  18: 'rutgers-the',
  19: 'dorotocephala',
  20: 'windowsapplication',
  21: '2015led',
  22: 'othersfocused',
  23: '429',
  24: 'benefitsresource',
  25: 'data-cabling',
  26: 'developmentfinancial',
  27: 'worker-therapist',
  28: 'reportsquality',
  29: 'solutionsdesigned',
  30: 'steward',
  31: 'teachingdecember',
  32: 'skava',
  33: 'aif',
  34: 'analysisconsultantsagile',
  35: '2016hillingdon',
  36: 'r2application',
  37: 'captioning',
  38: 'deliving',
  39: 't2',
  40: 'abi',
  41: 'compounddissolution',
  42: 'syspro',
  43: 'emphasize',
  44: "'student",
  45: 'arts-political',
  46: 'educationgrade',
  47: 'sappers

In [120]:
data_all = []
if not isinstance(data_normalized, type(None)):
    for token in data_normalized:
        if not isinstance(token, type(None)):
            for sentence in token:
                if not isinstance(sentence, type(None)):
                    for s in sentence:
                        data_all.append(s)

In [121]:
data_all

['freelance',
 'copy',
 'editor',
 'of',
 '100',
 'research',
 'and',
 'commercialisation',
 'intern',
 'hold',
 'full',
 'uk',
 'driving',
 'license',
 'volunteer',
 'campsite',
 'attendant',
 'light-scattering',
 'technique',
 'inkjet',
 'printing',
 'nmr',
 'size-exclusion',
 'chromatograhy',
 'uv-visible',
 'spectroscopy',
 'fluorescence',
 'post-doctoral',
 'research',
 'associate',
 'team',
 'leader',
 'night',
 'manager',
 'relief',
 'manager',
 'supervisor',
 'duty',
 'manager',
 'assistant',
 'manager',
 'postdoctoral',
 'research',
 'assistant',
 'subharmonic',
 'function',
 'on',
 'graph',
 'differential',
 'equation',
 'project',
 'within',
 'the',
 'course',
 'of',
 'programming',
 'expertise',
 'and',
 'consulting',
 'delivering',
 'result',
 'preparing',
 'concise',
 'and',
 'logically-written',
 'material',
 'ability',
 'to',
 'organize',
 'and',
 'communicate',
 'idea',
 'effectively',
 'in',
 'oral',
 'presentation',
 'to',
 'small',
 'and',
 'large',
 'group',
 'meet

In [110]:
a = '''sdasd asdasd sdasdas'''

s = a.split()

s = ['<s>'] + s + ['</s>']
s

['sdasd', 'asdasd', 'sdasdas']

In [ ]:
# def build_vocab(data_word_tokenized, target):
#     vocab = set()
#     for token in data_word_tokenized:
#         for sent in token:
#             if not isinstance(sent, type(None)):
#                 for word in sent:
#                     vocab.add(word)
            
#     if target:
#         w2i = {w: np.int32(i+2) for i, w in enumerate(vocab)}
#         w2i['<s>'], w2i['</s>'] = np.int32(0), np.int32(1)
#     else:
#         w2i = {w: np.int32(i) for i, w in enumerate(vocab)}

#     return w2i

In [123]:
# data_normalized = data_word_tokenized.map(lambda l: map(lambda wl: map(lambda w: nltk.stem.PorterStemmer.NLTK_EXTENSIONS(w) if w in wl and not isinstance(w, type(None)) else wl.remove(w), wl), l))

In [131]:
ree = re.compile(r'(\'\w)')
wl = list()

for i in data_normalized:
    for j in i:
        if not isinstance(j, type(None)):
            for k in j:
                for l in k:
            

TypeError: 'str' object is not callable

In [ ]:
ree = re.compile(r'(\'\w)')
wl = list()

for i in normalize(data_word_tokenized,**tokenizer_prefs):
    for j in i:
        if not isinstance(j, type(None)):
            for k in j:
                if not isinstance(k, type(None)):
                    if re.match(ree, k):
                        ree.sub('', k)
                    if len(k.strip().strip('.').strip(',')) > 1:
                        wl.append((k))
            

In [101]:
def removeNoneTypes(lst):
    return [i for i in lst if type(i) is not type(None)]

In [105]:
def normalize(data_word_tokenized, **kwargs):
    tokenizer = kwargs['tokenizer']
    normalizer = kwargs['token_format']
    spelling = kwargs['spell_correct']
    
    data_normalized = data_word_tokenized.map(lambda l: map(lambda wl: removeNoneTypes(wl), l))
    data_normalized = data_normalized.map(lambda l: map(lambda w: w.singularize(), l))
    
    # filter out 'bad' words, normalize good ones
    # w if w not in self.stopWords else wl.remove(w)
    data_normalized = data_normalized.map(lambda l: map(lambda wl: map(lambda w: wl.remove(w) if w in stopWords else w, wl), l))
    data_normalized = data_normalized.map(lambda l: map(lambda wl: map(lambda w: wl.remove(w) if w == '\'s' else w, wl), l))
    data_normalized = data_normalized.map(lambda l: map(lambda wl: map(lambda w: wl.remove(w) if w == '\'d' else w, wl), l))
 
     # remove tokens with length 1
#     ree = re.compile(r'(\'\w)')
#     rlen = len(data_normalized)
#     tmp = data_normalized.copy()
# #     tmp.reset_index(drop=True)
#     for indx in tmp.index:
         wl_coll = list()
         for i in normalize(data_word_tokenized,**tokenizer_prefs):
            for j in i:
                if not isinstance(j, type(None)):
                    for k in j:
                        if not isinstance(k, type(None)):
                            if re.match(ree, i):
                                ree.sub('', i)
                            if len(i.strip().strip('.').strip(',')) > 1:
                                wl.append((i))
                    wl_coll.append(WordList(wl))
            data_normalized[indx] = wl_col
            del tmp

    # stemming
    data_normalized = data_normalized.map(lambda l: map(lambda wl: map(lambda w: nltk.stemmer.stem(w) if w in wl and not isinstance(w, types.NoneType) else wl.remove(w), wl), l))

    data_word_tokenized= tokenize_words('value_char', **tokenizer_prefs)
    
    return data_word_tokenized

In [108]:
stopWords = stopwords.words('english')
tokenizer  =   RegexpTokenizer(pattern=r'\w+')
stemmer    =   nltk.stem.PorterStemmer.NLTK_EXTENSIONS
lemmatize  =   nltk.WordNetLemmatizer()

In [107]:
normalize(data_word_tokenized,**tokenizer_prefs)

0        <map object at 0x7fbaa7e44550>
1        <map object at 0x7fbaa7e445c0>
2        <map object at 0x7fbaa7e44630>
3        <map object at 0x7fbaa7e446d8>
4        <map object at 0x7fbaa7e44780>
6        <map object at 0x7fbaa7e44828>
7        <map object at 0x7fbaa7e448d0>
8        <map object at 0x7fbaa7e44978>
9        <map object at 0x7fbaa7e44a20>
10       <map object at 0x7fbaa7e44ac8>
11       <map object at 0x7fbaa7e44b70>
12       <map object at 0x7fbaa7e44c18>
13       <map object at 0x7fbaa7e44cc0>
14       <map object at 0x7fbaa7e44d68>
18       <map object at 0x7fbaa7e44e10>
19       <map object at 0x7fbaa7e44eb8>
21       <map object at 0x7fbaa7e44f60>
22       <map object at 0x7fbaa7e46048>
23       <map object at 0x7fbaa7e460f0>
24       <map object at 0x7fbaa7e46198>
25       <map object at 0x7fbaa7e46240>
29       <map object at 0x7fbaa7e462e8>
30       <map object at 0x7fbaa7e46390>
31       <map object at 0x7fbaa7e46438>
32       <map object at 0x7fbaa7e464e0>


In [108]:
ree = re.compile(r'(\'\w)')
wl = list()

for i in normalize(data_word_tokenized,**tokenizer_prefs):
    for j in i:
        if not isinstance(j, type(None)):
            for k in j:
                if not isinstance(k, type(None)):
                    if re.match(ree, k):
                        ree.sub('', k)
                    if len(k.strip().strip('.').strip(',')) > 1:
                        wl.append((k))
            

In [109]:
wl

['mitcham',
 'guest',
 'lecturer',
 'crewe',
 'pencari',
 'kerja',
 'an',
 'ambitious',
 'and',
 'hardworking',
 'individual',
 'who',
 'is',
 'motivated',
 'by',
 'challenge',
 'and',
 'is',
 'passionate',
 'to',
 'be',
 'successful',
 'inventor',
 'on',
 'four',
 'patents',
 'excellent',
 'communicator',
 'strong',
 'planning',
 'organisational',
 'problem',
 'solving',
 'skills',
 'with',
 'the',
 'ability',
 'to',
 'successfully',
 'analyse',
 'and',
 'assimilate',
 'large',
 'amounts',
 'of',
 'complex',
 'and',
 'disparate',
 'information',
 'good',
 'time',
 'management',
 'enjoys',
 'working',
 'under',
 'pressure',
 'and',
 'to',
 'deadlines',
 'either',
 'individually',
 'or',
 'as',
 'part',
 'of',
 'team',
 'conversational',
 'german',
 'part',
 'time',
 'lecturing',
 'and',
 'lab',
 'demonstrator',
 'project',
 'evaluation',
 'and',
 'responsible',
 'innovation',
 'intern',
 'computer',
 'skills-',
 'good',
 'knowledge',
 'of',
 'it',
 'and',
 'confident',
 'in',
 'using',

In [ ]:


def normalize_data(**kwargs):
    tokenizer = kwargs['tokenizer']
    normalizer = kwargs['token_format']
    spelling = kwargs['spell_correct']
    
#     data = pd.DataFrame(['asc','asda','asdasdasd'], columns=['value_char'])
    
    # singularize tokens
#     data = data[prefix].map(lambda l: map(lambda w: w.singularize(), l))

    # Spell correct flag
    # REALLY SHOULD NEVER BE USED
#     if spelling:
#         print("Spell Correction Invoked.....")
#         data[prefix] = data[prefix].map(lambda l: map(lambda wl: map(lambda w: w.correct(), wl), l))
#         print(data[prefix].map(lambda l: map(lambda w: type(w), l)))

    # filter out 'bad' words, normalize good ones
    # w if w not in self.stopWords else wl.remove(w)
    aa = cleanestes.map(lambda l: map(lambda wl: map(lambda w: wl.remove(w) if w in stopWords else w, wl), l))
    aa = aa.map(lambda l: map(lambda wl: map(lambda w: wl.remove(w) if w == '\'s' else w, wl), l))
    aa = aa.map(lambda l: map(lambda wl: map(lambda w: wl.remove(w) if w == '\'d' else w, wl), l))

    # remove tokens with length 1
#     ree = re.compile(r'(\'\w)')
#     rlen = len(data)
#     tmp = data[prefix].copy()
#     for index in range(0,rlen):
#         wl_coll = list()
#         for lst in tmp[index]:
#             wl = list()
#             for word in lst:
#                 if not isinstance(word, types.NoneType):
#                     if re.match(ree, word):
#                         ree.sub('', word)
#                     if len(word.strip().strip('.').strip(',')) > 1:
#                         wl.append((word))
#             wl_coll.append(WordList(wl))
#         data[index] = wl_coll
#     del tmp

    # remove via regexp c'c pattern

    # Stemming or lemmatization of tokens    
    if normalizer == 'stem':
        aa = aa.map(lambda l: map(lambda wl: map(lambda w: stemmer.stem(w) if w in wl and not isinstance(w, types.NoneType) else wl.remove(w), wl), l))
#     elif normalizer == 'lemma':
#         data[prefix] = data[prefix].map(lambda l: map(lambda wl: map(lambda w: w.lemmatize(), wl), l))
#     elif normalizer == 'None':
#         pass

    aa = aa.map(lambda l: map(lambda wl: map(Word, wl), l))
#     data[prefix] = data[prefix].map(lambda l: map(WordList, l))
    
    return aa

In [ ]:
normalize_data(**tokenizer_prefs)